# 03.05 - Hierarchical Indexing

Beside one-dimensional (<code>Series</code>) and two-dimensional (<code>Dataframe</code>) data, Pandas allows us to store also multidimensional data, i.e. data that is indexed by more than two keys.

**Hierarchical indexing** or multi-indexing can do just this, incorporating multiple index levels within a single index.

In [1]:
import pandas as pd
import numpy as np

### A Multiply Indexed Series

Let's start by examining how to represent two-dimensional data within a one-dimensional Series:

In [24]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

### Pandas MultiIndex

In [25]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

If we re-index our series with this MultiIndex, we see the hierarchical representation of the data:

In [26]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Now we can simply use Pandas slicing notation to access the data:

In [27]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

The <code>unstack()</code> method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:

In [28]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


As you would expect, <code>stack</code> does exactly the opposite:

In [29]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Then why bother with hierarchical indexing? Well, it turns out that multi-indexing is particularly helpful to add extra dimensions to our data, which is as easy as adding another column to our Dataframe:

In [32]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Additionally, ufuncs work with hierarchical indexing as well:

In [33]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### Methods of MultiIndex Creation

The simplest way is to pass a list of two or more index arrays to the constructor:

In [34]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.420059  0.912932
  2  0.391889  0.622146
b 1  0.067290  0.123013
  2  0.367496  0.822409

Additionally, for appropriately formatted dicts, Pandas will proceed with hierarchical indexing by default:

In [35]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors

There are several ways to construct the <code>MultiIndex</code> explicitely:

In [36]:
# index values separate
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [37]:
# value, index tuples
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [38]:
# cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Or we can explicit both levels and labels:

In [39]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])